### **Data Engineer Intern Practical Exam**

*Gayandee Rajapaksha - gayandeethathsiluni@gmail.com*

#### **Part 1: Data Preparation**

In [4]:
!pip install pandas sqlalchemy pymysql

**I installed pandas for the purpose of reading files and handling tables . Used sqlalchemy to for the connection with Database and pymysql to help with connecting using MySQL credentials.**

In [51]:
import pandas as pd
from sqlalchemy import create_engine, text

# Load data
customers_df = pd.read_csv('customers.csv')
orders_df = pd.read_csv('order.csv')

# Renamed 'created_at' to 'order_date' in orders_df
orders_df.rename(columns={'created_at': 'order_date'}, inplace=True)
orders_df.rename(columns={'id': 'order_id'}, inplace=True)
customers_df.rename(columns={'name': 'customer_name'}, inplace=True)

# Convert 'order_date' to datetime and keep only the date part
orders_df['order_date'] = pd.to_datetime(orders_df['order_date']).dt.date

# Keep only the necessary columns for the database
customers_df = customers_df[['customer_id', 'customer_name']]
orders_df = orders_df[['order_id', 'customer_id', 'total_amount', 'order_date']]

# Create database connection
cnx = create_engine('mysql+pymysql://root<password>localhost:3306/delivergate')

# Create a connection object
conn = cnx.connect()

# Temporarily disable foreign key checks
conn.execute(text("SET FOREIGN_KEY_CHECKS = 0"))

# Drop the 'customers' and 'orders' tables if they exist
conn.execute(text("DROP TABLE IF EXISTS orders"))
conn.execute(text("DROP TABLE IF EXISTS customers"))

# Re-enable foreign key checks
conn.execute(text("SET FOREIGN_KEY_CHECKS = 1"))

# Import data into MySQL
customers_df.to_sql('customers', con=cnx, if_exists='replace', index=False)
orders_df.to_sql('orders', con=cnx, if_exists='replace', index=False)

# Close the connection
conn.close()


**This code reads a file named customers.csv, order.csv and puts all that data into DataFrames. Then it creates a connection to my MySQL database called delivergate, using the code that includes my username and password.**

**Then renamed and filter out the columns which was asked on the question. I believe the reason is focusing the target columns rather than unnecessary columns which can  effect accuracy in future questions.**

**Then I temporarily disabled foreign key checks to safely drop the existing customers and orders tables in the MySQL database before importing new data from the DataFrames, ensuring that there are no conflicts or errors during the import process.**

In [52]:
# Query to check data in customers table and store it in a DataFrame
customers_check = pd.read_sql('SELECT * FROM customers', con=cnx)
customers_df = pd.DataFrame(customers_check)

# Query to check data in orders table and store it in a DataFrame
orders_check = pd.read_sql('SELECT * FROM orders', con=cnx)
orders_df = pd.DataFrame(orders_check)

In [54]:
customers_df

,customer_id,customer_name
0,8,customer three
1,9,Shafran Naizer
2,10,Isuri Liyanage
3,12,Shafran Naizer
4,13,sss ss
...,...,...
1109,1270,Alex Johnson
1110,1271,Dinuka Kaveen Dominguhewa
1111,1272,K L
1112,1273,fdv dfvfd


In [53]:
orders_df

,order_id,customer_id,total_amount,order_date
0,13392,1251.0,425,2024-10-14
1,13393,1251.0,1650,2024-10-14
2,13394,468.0,1365,2024-10-14
3,13395,1251.0,525,2024-10-14
4,13396,468.0,525,2024-10-14
...,...,...,...,...
8112,814,58.0,450000,2022-03-03
8113,826,58.0,125000,2022-03-03
8114,830,58.0,260000,2022-03-03
8115,845,58.0,125000,2022-03-04


These steps I did to ensure that I have successfully loaded the needed data from the database

#### **Part 2: Streamlit App Setup & Part 3: Data Analysis**

In [34]:
!pip install streamlit

In [24]:
!pip install scikit-learn

**I used Streamlit to cover both Part 2 and Part 3 using navigations in the sidebar itself. For the documentation purpose I added the code here. The code can be run from the python file called StreamlitML.py**

In [40]:
import pandas as pd
import numpy as np
import streamlit as st
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt


# This code shows the title and layout configuration.
st.set_page_config(page_title="Customer Purchase", layout="centered")

# Thus creates a connection to the MySQL database as before
engine = create_engine('mysql+pymysql://root:<password>@localhost:3306/delivergate')

# These SQL queries to select all data from the customers and orders tables.
customers_query = "SELECT * FROM customers;"
orders_query = "SELECT * FROM orders;"
customers_df = pd.read_sql(customers_query, con=engine)
orders_df = pd.read_sql(orders_query, con=engine)

# Used sidebar for navigation and used a radio button allows users to switch between 
# the "Dashboard" and "Data Analysis" pages which completes 2nd and 3rd parts respectively.
st.sidebar.header("Navigation")
page = st.sidebar.radio("Select Page", ("Dashboard", "Data Analysis"))

if page == "Dashboard":
    # 01 Sidebar Filters
    st.sidebar.header("Filters")

    # Converted the order_date column in the orders DataFrame to a datetime format. 
    # This feature engineering approch will effect for the accuracy.
    orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])
    
    # Get the min and max dates
    min_date = orders_df['order_date'].min()
    max_date = orders_df['order_date'].max()

    # Ensure min_date and max_date are not None
    if min_date is None or max_date is None:
        st.error("No valid dates available in the dataset.")
    else:
        # Get the date range from the user
        date_range = st.sidebar.date_input("Select Date Range", [min_date.date(), max_date.date()])

        # Validated that the date range selected is in the correct format (a list or tuple with two items)
        if isinstance(date_range, (list, tuple)) and len(date_range) == 2:
            start_date, end_date = date_range

            start_date = pd.to_datetime(start_date)
            end_date = pd.to_datetime(end_date)
        else:
            st.error("Please select a valid date range.")

    # Total amount slider
    min_amount, max_amount = st.sidebar.slider("Select Total Amount Range",
                                                float(0),
                                                float(orders_df['total_amount'].max()),
                                                (float(0), float(orders_df['total_amount'].max())))

    # Customer order count dropdown
    order_count = st.sidebar.number_input("Minimum Number of Orders", min_value=1, value=5)

    # Filtered the orders DataFrame to include only the rows that match the selected date range and total amount criteria.
    filtered_orders = orders_df[(orders_df['order_date'] >= start_date) & 
                                 (orders_df['order_date'] <= end_date) &
                                 (orders_df['total_amount'].between(min_amount, max_amount))]

    #Counting how many orders each customer has made and filtering the orders DataFrame 
    # to include only those customers who meet the minimum order count specified by the user.
    customer_counts = filtered_orders['customer_id'].value_counts()
    filtered_customers = customer_counts[customer_counts >= int(order_count)].index
    filtered_orders = filtered_orders[filtered_orders['customer_id'].isin(filtered_customers)]

    # 02 Main dashboard
    st.title("Customer Orders Dashboard")

    # Display filtered data
    st.subheader("Filtered Orders Data")
    st.dataframe(filtered_orders)

    # Metrics
    total_revenue = filtered_orders['total_amount'].sum()
    unique_customers = filtered_orders['customer_id'].nunique()
    total_orders = filtered_orders.shape[0]

    st.subheader("Key Metrics")
    st.write(f"Total Revenue: ${total_revenue:.2f}")
    st.write(f"Number of Unique Customers: {unique_customers}")
    st.write(f"Total Orders: {total_orders}")

    # Top 10 customers by total revenue
    top_customers = filtered_orders.groupby('customer_id')['total_amount'].sum().nlargest(10)
    st.subheader("Top 10 Customers by Total Revenue")
    st.bar_chart(top_customers)

    # Total revenue over time (grouped by month)
    revenue_over_time = filtered_orders.groupby(pd.Grouper(key='order_date', freq='M'))['total_amount'].sum()
    st.subheader("Total Revenue Over Time")
    st.line_chart(revenue_over_time)

elif page == "Data Analysis":
    st.title("Data Analysis Section")
    
    # Setted a color palette for the visualizations
    sns.set_palette("Set2")

    # Summarized the orders data to get the total number of orders and revenue per customer.
    order_summary = orders_df.groupby('customer_id').agg(
        total_orders=('order_id', 'count'),
        total_revenue=('total_amount', 'sum')
    ).reset_index()

    # Create the target variable called repeat purchaser
    order_summary['repeat_purchaser'] = np.where(order_summary['total_orders'] > 1, 1, 0)

    # Features(x) and target variable(y)
    X = order_summary[['total_orders', 'total_revenue']]
    y = order_summary['repeat_purchaser']

    # Validation check for sufficient data
    if len(X) < 10:
        st.error("Not enough data to train the model.")
    else:
        # Split data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the logistic regression model
        model = LogisticRegression()
        model.fit(X_train, y_train)

        # Predictions
        y_pred = model.predict(X_test)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # Display results
        st.subheader("Logistic Regression Model Results")
        st.markdown(f"<h3 style='color: green;'>Model Accuracy: {accuracy:.2f}</h3>", unsafe_allow_html=True)

        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        fig, ax = plt.subplots(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Repeat', 'Repeat'], yticklabels=['Not Repeat', 'Repeat'], ax=ax)
        plt.title('Confusion Matrix', fontsize=16)
        plt.xlabel('Predicted', fontsize=14)
        plt.ylabel('Actual', fontsize=14)
        plt.tight_layout()
        st.pyplot(fig)

        st.subheader("Classification Report:")

        # Generated the classification report as a DataFrame
        report = classification_report(y_test, y_pred, output_dict=True)
        report_df = pd.DataFrame(report).transpose()

        # Displayed the report as a grid in Streamlit
        st.dataframe(report_df)

        # Total Orders Distribution
        orders_chart = alt.Chart(order_summary).mark_bar().encode(
            x=alt.X('total_orders:Q', bin=True, title='Total Orders'),
            y=alt.Y('count():Q', title='Frequency'),
            color=alt.Color('repeat_purchaser:N', scale=alt.Scale(domain=[0, 1], range=['red', 'green']), title='Repeat Purchaser')
        ).properties(
            width=600,
            height=400,
            title='Total Orders Distribution by Repeat Purchasers'
        )

        st.altair_chart(orders_chart, use_container_width=True)

        # Total Revenue Distribution
        revenue_chart = alt.Chart(order_summary).mark_bar().encode(
            x=alt.X('total_revenue:Q', bin=True, title='Total Revenue'),
            y=alt.Y('count():Q', title='Frequency'),
            color=alt.Color('repeat_purchaser:N', scale=alt.Scale(domain=[0, 1], range=['red', 'green']), title='Repeat Purchaser')
        ).properties(
            width=600,
            height=400,
            title='Total Revenue Distribution by Repeat Purchasers'
        )

        st.altair_chart(revenue_chart, use_container_width=True)


        # By this we can input new data for prediction
        st.subheader("Predict New Customer's Repeat Purchase Status")
        new_orders = st.number_input("Total Orders", min_value=1)
        new_revenue = st.number_input("Total Revenue", min_value=0.0)

        if st.button("Predict"):
            new_data = np.array([[new_orders, new_revenue]])
            prediction = model.predict(new_data)
            st.markdown(f"<h4 style='color: {'red' if prediction[0] == 0 else 'green'};'>Repeat Purchaser Status (1 = Yes, 0 = No): {prediction[0]}</h4>", unsafe_allow_html=True)


# Run the app
if __name__ == '__main__':
    st.write("Gayandee Rajapaksha!")


**In my Streamlit Application, I created a user-friendly interface to analyze customer purchase data effectively. I established a connection to a MySQL database, allowing me to retrieve the relevant data for analysis.** 

**I made the decision to include a sidebar for navigation, which provides users with the ability to switch between the "Dashboard" and "Data Analysis" pages seamlessly.** 

.

**On the "Dashboard," (Part 2) I implemented filters that let users specify a date range, total spending amount, and the minimum number of orders, enabling them to view only the most relevant data.** 

**This is why I used visualizations such as bar charts for top customers and line charts for revenue trends; they help in presenting the data intuitively, allowing for quicker insights into customer purchasing behavior.**

.


**In the "Data Analysis"(Part 3) section, I focused on predicting whether customers would be repeat purchasers based on their total orders and revenue.** 

**I created a logistic regression model for this purpose, as it effectively predicts binary outcomes. I included validation checks to ensure there was sufficient data for training the model; if not, I alerted the user to the lack of data.** 

**I chose to display the accuracy of the model and provided a confusion matrix alongside a classification report, as these metrics are crucial for understanding model performance. Furthermore, I used Altair for visualizing the distributions of total orders and revenue among repeat purchasers, which I believe makes the insights more accessible.**

**Additionally, I enabled a feature that allows users to input new data for prediction, making the application not only informative but also practical for real-time decision-making regarding customer retention strategies.**

.

**In this classification report, I believe it summarizes the performance of my model quite effectively. The precision, recall, and F1-score for both classes, 0 and 1 are all perfect, sitting at 1. This indicates that my model has made no mistakes in classifying either of the classes, which is a remarkable achievement in any classification task.**

**The precision of 1 for both classes means that whenever my model predicted a class (either 0 or 1), it was correct. Similarly, the recall of 1 signifies that my model successfully identified all instances of each class in the dataset. The F1-score, which is the harmonic mean of precision and recall, also stands at 1, indicating a balanced performance.**

**I also note that the accuracy of the model is listed as 1, meaning that 100% of the predictions were correct. The support column shows how many actual occurrences there were for each class in the dataset, which were 98 for class 0 and 35 for class 1, totaling 133 instances. The macro and weighted averages confirm that my model performs consistently across different metrics, which reflects its robustness. Overall, these results indicate that my model is highly effective on this simple dataset.**

In [ ]:
!streamlit run StreamlitML.py